In [1]:
import jax.numpy as jnp

In [10]:
DATA = 'meshes/'
examplemesh = 'box.obj'

def objloader(folder, fname):
    V = []
    F = []
    with open(folder + fname, 'r') as f:
        lines = f.readlines()
        for l in lines:
            token = l.split(' ')[0]
            if token == 'v':
                V.append(jnp.array([float(v) for v in l.split(' ')[1:]]))
            if token == 'f':
                F.append(jnp.array([int(f.split('/')[0]) - 1 for f in l.split(' ')[1:]]))
    V = jnp.array(V)
    F = jnp.array(F, dtype=jnp.int32)
    return V, F

def get_edge(V, e0, e1):
    return V[e1] - V[e0]

V, F = objloader(DATA, examplemesh)
print(F)

[[0 1 2]
 [1 0 3]
 [4 5 6]
 [5 4 7]
 [4 1 3]
 [1 4 6]
 [1 5 2]
 [5 1 6]
 [5 0 2]
 [0 5 7]
 [4 0 7]
 [0 4 3]]


## Coding 10.1

### Compute the uniformly weighted vertex normals given by

$
N_U = \sum\limits_i N_i
$

In [18]:
def normalize(v):
    return v / jnp.linalg.norm(v)

def compute_normal(e0, e1):
    return normalize(jnp.cross(e0, e1))

def compute_uniformly_weighted_normals(V, F):
    Nu = []
    for vi, v in enumerate(V):
        # Get neighbouring faces
        Nf = [fi for fi, f in enumerate(F) if vi in f]
        # Compute normals
        Nv = jnp.zeros(3)
        for fi in Nf:
            f = F[fi]
            Nv += compute_normal(V[f[1]] - V[f[0]], V[f[2]] - V[f[0]])
        Nu.append(normalize(Nv))
    return jnp.array(Nu)
        
compute_uniformly_weighted_normals(V, F)

DeviceArray([[-0.57735026,  0.57735026, -0.57735026],
             [ 0.57735026,  0.57735026,  0.57735026],
             [ 0.57735026,  0.57735026, -0.57735026],
             [-0.57735026,  0.57735026,  0.57735026],
             [-0.57735026, -0.57735026,  0.57735026],
             [ 0.57735026, -0.57735026, -0.57735026],
             [ 0.57735026, -0.57735026,  0.57735026],
             [-0.57735026, -0.57735026, -0.57735026]], dtype=float32)

## Coding 10.2

### Compute the vertex normal using face area weights

$
N_\nu = \sum\limits_i N_i
$